In [1]:
#importing the required libraries for python

import numpy as np
import matplotlib.pyplot as plt


In [41]:

def Rotation(theta, X):
    import math
    rad = theta*math.pi/180
    R = np.array([[math.cos(rad),0,-math.sin(rad)],
                  [0,1,0],
                  [math.sin(rad), 0, math.cos(rad)]])
    X_n = np.zeros(X.shape)
    for i in range(X.shape[0]):
        X_n[i] = R.dot(X[i])
    return X_n


In [42]:
#Settingg up parameters for NURBS surface

#Order of NURBS basis
p = 2
q = 1
r = 4

#Number of Basis function for NURBS
n = 3
m = 2
l = 9

In [43]:
#Defining the knot Vectot for NURBS function

Xi = np.array([0,0,0,1,1,1])
Eta = np.array([0,0,1,1])
Zeta = np.array([0,0,0,0,0,0.2,0.4,0.6,0.8,1,1,1,1,1])


In [44]:
#Control points array and weights 

Mod1 = np.array([[0,-20,0.5],[0,-20,0.5],[0,-20,0.5],[0,-20,0.5],[0,-20,0.5],[0,-20,0.5],[0,-20,0.5],[0,-20,0.5],[0,-20,0.5],
                    [-3,-20,0],[-3,-20,1],[0,-20,1],[3,-20,1],[3,-20,0],[3,-20,-1],[0,-20,0],[-3,-20,-1],[-3,-20,0]])

Mod2 = np.array([[0,0,0.5],[0,0,0.5],[0,0,0.5],[0,0,0.5],[0,0,0.5],[0,0,0.5],[0,0,0.5],[0,0,0.5],[0,0,0.5],
                [-3.75,0,0],[-3.75,0,1.25],[0,0,1.25],[3.75,0,1.25],[3.75,0,0],[3.75,0,-1.25],[0,0,0],[-3.75,0,-1.25],[-3.75,0,0]])

Mod3 = np.array([[0,20,0.5],[0,20,0.5],[0,20,0.5],[0,20,0.5],[0,20,0.5],[0,20,0.5],[0,20,0.5],[0,20,0.5],[0,20,0.5],
             [-2.25,20,0],[-2.25,20,0.75],[0,20,0.75],[2.25,20,0.75],[2.25,20,0],[2.25,20,-0.75],[0,20,0],[-2.25,20,-0.75],[-2.25,20,0]])

ctrlpts = np.append(Mod1, Rotation(5,Mod2),axis=0)
ctrlpts = np.append(ctrlpts,Rotation(10,Mod3),axis=0)

w = np.array([1,0.75,1,0.75,1,1,5,1,1,
              1,0.75,1,0.75,1,1,5,1,1,
              1,0.75,1,0.75,1,1,5,1,1,
              1,0.75,1,0.75,1,1,5,1,1,
              1,0.75,1,0.75,1,1,5,1,1,
              1,0.75,1,0.75,1,1,5,1,1])


In [5]:
#finding the total number of elements

def NumOfEle(Xi):
    n = 0
    for i in range((len(Xi)-1)):
        if Xi[i] != Xi[i+1]:
            n = n+1
    return n

print(NumOfEle(Xi))
print(NumOfEle(Eta))

1
2


In [45]:
#Converting the local indexing to the global indexing

def IEN2D(i,j,m):
    return (m*(i-1) + j)

def IEN3D(i,j,k,l,m):
    return ((l*m)*(i-1)+l*(j-1)+k)

In [56]:
#Finding the basis functions for the NURBS

def coxDeBoor(zeta, i, d, knot_vect):
    '''
    zeta - value of the point we need to find
    i - i'th basis function
    d - order of basis function
    knot_vect - knot vector for NURBS    
    '''
    if d==0:
        if knot_vect[i] <= zeta and zeta <= knot_vect[i+1]:
            return 1
        return 0
    
    D1 = knot_vect[i+d] - knot_vect[i]
    D2 = knot_vect[i+d+1] - knot_vect[i+1]
    
    E1 = 0
    E2 = 0
    
    if D1 > 0:
        E1 = (zeta - knot_vect[i])/D1 * coxDeBoor(zeta, i, d-1, knot_vect)
    
    if D2 > 0:
        E2 = (knot_vect[i+d+1] - zeta)/D2 * coxDeBoor(zeta, i+1, d-1, knot_vect)
        
    return E1+E2


In [57]:
#Computing the Beizer extraction operator 'C' for different elements

Xi = [0,0,0,0,1,2,3,4,4,4,4]

p = 3

def Beizer_extraction(U, p):
    a = p+1
    o = p+1
    b = a+1
    nb = 0
    C = np.zeros((NumOfEle(U),o,o))
    C[0] = np.identity(o)
    
    while b < len(U) and nb < (NumOfEle(U) - 1):
        C[nb+1] = np.identity(o)
        i = b
        mult = 1
        #Counting the multiplicity of the knot at location b
        while b<len(U) and (U[b] == U[b-1]):
            b = b+1
            mult = b-i+1
        
        if mult < p:
            numerator = U[b-1] - U[a-1]
            alphas = np.zeros((1,p-mult))
            for j in range(p,mult,-1):
                alphas[0,j-mult-1] = numerator/(U[a+j-1]-U[a-1])
            r = p - mult
            for j in range(1,r+1):
                save = r-j+1
                s = mult+j
                for k in range(p+1,s,-1):
                    alpha = alphas[0,k-s-1]
                    C[nb,:,k-1] = alpha*C[nb,:,k-1] + (1-alpha)*C[nb,:,k-2]

                if b < len(U):
                    C[nb+1,save-1:j+save,save-1] = C[nb, p-j:p+1, p]
        
        nb = nb+1  
        if b < len(U):
            a = b
            b = b+1
            
    return C



In [3]:
def Bernstein_basis(a, p, xi):
    if a == 1 and p == 0:
        return 1
    if (a < 1) or (a > p+1):
        return 0
    
    b = (1 - xi)*Bernstein_basis(a, p-1, xi) + xi*Bernstein_basis(a-1, p-1, xi)
    
    return b


In [ ]:
def derivative_Bernstein(a, p, xi):
    return a*(Bernstein_basis(a-1, p-1, xi) - Bernstein_basis(a-1, p-1, xi))


In [58]:
def BasisFunction(xi, eta, zeta):
    N = np.zeros(l*m*n)
    Tot_weight = 0
    for i in range(n):
        for j in range(m):
            for k in range(l):
                N[IEN3D(i,j,k,l,m)] = coxDeBoor(xi, i, p, Xi)*coxDeBoor(eta, j, q, Eta)*coxDeBoor(zeta, k, r, Zeta)*w[IEN3D(i,j,k,l,m)]
                Tot_weight = Tot_weight + N[IEN3D(i,j,k,l,m)
    
    return (N/Tot_weight)



In [ ]:
def Bezier_Basis(zeta, a, p):
    if p = 0 and a = 1:
        return 1
    if a<1 or a>p+1:
        return 0
    
    r = (1-zeta)*Bezier_Basis(zeta,a,p-1) + zeta*Bezier_Basis(zeta,a-1,p-1)
    
    return r    

In [ ]:
def knot_insertion(U, knot, ctrlpts):
    U_n = U.append(knot)
    U_n = sorted(U_n)
    for i in range(len(knot_Vect)-1):
        if knot_Vect[i]<=k and k < knot_Vect[i+1]:
            k = i 
    ctrlpts_n = 
